In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Regularization Data Setup

In [34]:
df = pd.read_csv('./data/advertising.csv')
X = df.drop('sales', axis=1)
y = df['sales']

In [35]:
from sklearn.preprocessing import PolynomialFeatures

In [36]:
poly_converter = PolynomialFeatures(degree=3, include_bias=False)

In [37]:
poly_features = poly_converter.fit_transform(X)

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [40]:
from sklearn.preprocessing import StandardScaler

In [41]:
scaler = StandardScaler()

In [42]:
# first we need to fit using ONLY the training data!
scaler.fit(X_train)

StandardScaler()

In [43]:
# ... now we can transform both training and testing datasets
X_train = scaler.transform(X_train)

In [44]:
X_test = scaler.transform(X_test)

### Ridge Regression

In [45]:
from sklearn.linear_model import Ridge

In [46]:
# help(Ridge)

In [47]:
ridge_model = Ridge(alpha=10)

In [48]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [49]:
test_predictions = ridge_model.predict(X_test)

In [50]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [51]:
MAE = mean_absolute_error(y_test, test_predictions)

In [52]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [53]:
MAE

0.5774404204714177

In [54]:
RMSE

0.894638646131968

In [55]:
# ridge cross validation
from sklearn.linear_model import RidgeCV

In [69]:
# pass in a tuple of alpha values instead
# cv = k number of folds

a = (0.1, 1.0, 10.0)
# a = (0.01, 0.05, 0.1, 0.2, 0.4, 0.8, 1.0, 10.0)
ridgecv_model = RidgeCV(alphas=a, 
                        scoring='neg_mean_absolute_error')

In [70]:
# only using the training set to fit the model
ridgecv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [71]:
ridgecv_model.alpha_

0.1

In [61]:
# What was this using to choose the best-performing alpha?
from sklearn.metrics import SCORERS

In [62]:
# these are the errors we can use as our score for the error calculation
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [72]:
test_predictions = ridgecv_model.predict(X_test)

In [73]:
MAE = mean_absolute_error(y_test, test_predictions)

In [74]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions)) 

In [75]:
MAE

0.42737748843313855

In [76]:
RMSE

0.6180719926906028

In [77]:
ridgecv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

### LASSO Regression

In [79]:
from sklearn.linear_model import LassoCV

In [80]:
# use cross validation version of the Lasso model (LassoCV)

In [96]:
# does not converge
# lassocv_model = LassoCV(eps=0.001, n_alphas=100, cv=5)  

# option 1: 
lassocv_model = LassoCV(eps=0.001, n_alphas=100, cv=5, max_iter=1000000)

# option 2:
# lassocv_model = LassoCV(eps=0.1, n_alphas=100, cv=5) 

In [97]:
lassocv_model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=1000000)

In [98]:
lassocv_model.alpha_

0.004943070909225827

In [99]:
test_predictions = lassocv_model.predict(X_test)

In [100]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

print(MAE, "\n", RMSE)

0.43350346185900673 
 0.6063140748984039


In [102]:
# if we look at the coefficients, we see that LASSO is only considering some features 
# while getting similar accuracy to what we had before with Ridge Regression
lassocv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

### Elastic Net: L1 and L2 Regularization

In [103]:
from sklearn.linear_model import ElasticNetCV

In [105]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                             eps=0.001,
                             n_alphas=100,  # not the same alpha as in the course notes
                             max_iter=1000000)

In [106]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [108]:
elastic_model.l1_ratio_

1.0

In [109]:
elastic_model.alpha_

0.004943070909225827

In [110]:
test_predictions = elastic_model.predict(X_test)

In [111]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

print(MAE, "\n", RMSE)

0.43350346185900673 
 0.6063140748984039
